In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import transforms, models
from torch.utils.data import DataLoader, random_split

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
model = models.resnet18(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [1]:
# Set hyperparameters
EPOCH = 10
BATCH_SIZE = 128
LR = 0.01

# Prepare dataset and preprocessing
transform_train = transforms.Compose([
    transforms.Resize(32),                  # Resize for ResNet
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))    # Normalization for grayscale
])

transform_test = transforms.Compose([
    transforms.Resize(32),                  # Resize for ResNet
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))    # Normalization for grayscale
])

# Load Fashion MNIST dataset
trainset = torchvision.datasets.FashionMNIST(root='../data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

testset = torchvision.datasets.FashionMNIST(root='../data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)


NameError: name 'transforms' is not defined

In [18]:
# Labels in Fashion MNIST
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot')

In [19]:
# Define the Residual Block and ResNet
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out = out + self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=10):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False),  # Adjusted for grayscale images
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.layer1 = self.make_layer(ResidualBlock, 64, 2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

def ResNet18():
    return ResNet(ResidualBlock)

net = ResNet18().to(device)

In [20]:
# Define loss function & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)

In [21]:
# Training loop
for epoch in range(EPOCH):
    print('\nEpoch: %d' % (epoch + 1))
    net.train()
    sum_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        # Forward & backward
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Calculate accuracy and loss
        sum_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print('[epoch:%d, iter:%d] Loss: %.03f | Acc: %.3f%% '
              % (epoch + 1, (i + 1), sum_loss / (i + 1), 100. * correct / total))

    # Test after each epoch
    print('Testing...')
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print('Test Accuracy: %.3f%%' % (100 * correct / total))

print('Training complete.')


Epoch: 1
[epoch:1, iter:1] Loss: 2.376 | Acc: 13.281% 
[epoch:1, iter:2] Loss: 2.303 | Acc: 21.094% 
[epoch:1, iter:3] Loss: 2.244 | Acc: 22.917% 
[epoch:1, iter:4] Loss: 2.192 | Acc: 22.656% 
[epoch:1, iter:5] Loss: 2.129 | Acc: 26.406% 
[epoch:1, iter:6] Loss: 2.053 | Acc: 29.036% 
[epoch:1, iter:7] Loss: 1.963 | Acc: 31.696% 
[epoch:1, iter:8] Loss: 1.895 | Acc: 34.277% 
[epoch:1, iter:9] Loss: 1.821 | Acc: 36.806% 
[epoch:1, iter:10] Loss: 1.762 | Acc: 37.969% 
[epoch:1, iter:11] Loss: 1.696 | Acc: 40.554% 
[epoch:1, iter:12] Loss: 1.630 | Acc: 42.513% 
[epoch:1, iter:13] Loss: 1.568 | Acc: 44.651% 
[epoch:1, iter:14] Loss: 1.527 | Acc: 45.926% 
[epoch:1, iter:15] Loss: 1.480 | Acc: 47.760% 
[epoch:1, iter:16] Loss: 1.444 | Acc: 49.219% 
[epoch:1, iter:17] Loss: 1.409 | Acc: 50.276% 
[epoch:1, iter:18] Loss: 1.372 | Acc: 51.389% 
[epoch:1, iter:19] Loss: 1.341 | Acc: 52.467% 
[epoch:1, iter:20] Loss: 1.318 | Acc: 53.203% 
[epoch:1, iter:21] Loss: 1.285 | Acc: 54.464% 
[epoch:1, it